# Lab 2: GPT from scratch

In this lab, you will dive into the inner workings of the GPT architecture. You will walk through a complete implementation of the architecture in PyTorch, instantiate this implementation with pre-trained weights, and put the resulting model to the test by generating text. At the end of this lab, you will understand the building blocks of the GPT architecture and how they are connected.

*Tasks you can choose for the oral exam are marked with the graduation cap 🎓 emoji.*

In [1]:
from dataclasses import dataclass

import torch
import torch.nn as nn

## Part 1: GPT architecture

GPT-2 was first described by [Radford et al. (2019)](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf). To faithfully implement the model, one needs to also read the earlier paper by [Radford et al. (2018)](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf). Another important source of information is the code released by OpenAI, which is available on GitHub ([link](https://github.com/openai/gpt-2)).

The GPT architecture is made up of a stack of Transformer blocks. Each block has two main parts: one handles multi-head self-attention, and the other is a feed-forward network. Before these parts do their work, their input undergoes layer normalisation, and residual connections are added to help the model learn more effectively. The input to the architecture is a sequence of token IDs; these are turned into embeddings and augmented with information about the absolute position of each token in the sequence. The output layer converts the internal representations into logit scores for every token in the vocabulary.

### Model configuration

[Radford et al. (2019)](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf) present four increasingly larger GPT models based on the same architecture. Here, we will implement the smallest of these, characterised by the following hyperparameters:

In [2]:
@dataclass
class Config:
    n_vocab: int = 50_257
    n_ctx: int = 1024
    n_embd: int = 768
    n_head: int = 12
    n_layer: int = 12

#### 🎈 Task 2.01: Model configuration

Explain the purpose of these hyperparameters. In particular, where does the number 50,257 come from?

- `n_vocab` is the numbner of unique tokens, the number 50_257 came from the tokenizer when the model was trained.</br>
- `n_ctx` is the context length which is the number of tokens that the model can process at once. </br>
- `n_embd` the number of dimensions for the embedded vectors, basically when the tokens are converted into vectors. </br>
- `n_head` the number of multihead self attention (MHA).</br>
- `n_layer` The number of transformer blocks taht contains MHA, FFN.

### GELU activation function

We start by implementing the feed-forward network. This is a standard two-layer network with a Gaussian Error Linear Unit (GELU) activation function ([Hendrycks and Gimpel, 2016](https://doi.org/10.48550/arXiv.1606.08415)).

The GELU is a smooth version of the rectified linear unit (ReLU) that weights inputs by their value under the cumulative distribution function of the standard Gaussian. This function is commonly denoted by $\Phi$. For example, $\text{GELU}(0{.}5) = 0{.}5 \cdot \Phi(0{.}5) \approx 0{.}5 \cdot 0{.}6915 = 0{.}3457$ because approximately 69.15% of normally distributed data lies to the left of $0{.}5$.

When GPT-2 was released, computing the GELU exactly was expensive, and the released code therefore used an approximation originally presented by [Page (1977)](https://doi.org/10.2307/2346872). We follow suit here, as we want to create a replica of the original model. However, it is worth mentioning that PyTorch now offers an exact implementation of the GELU so fast that using an approximation is unnecessary.

In [3]:
def gelu(x):
    return 0.5 * x * (1 + torch.tanh((2 / torch.pi) ** 0.5 * (x + 0.044715 * x**3)))

#### 🎓 Task 2.02: Mathematical properties of the GELU

Find the minimal output value of the (approximated) GELU and the input value for which it yields that output. Use a service such as [WolframAlpha](https://www.wolframalpha.com/) for the necessary derivations. What are the main differences between the GELU and the ReLU?

min{0.5 x (1 + tanh((2/π)^0.5 (x + 0.044715 x^3)))}≈-0.170041 at x≈-0.752461

The difference between GELU and ReLU:
The way they both are visualized since ReLU shows a a convergence at 0 where values less than 0 are non differentiable, meanwhile GELU shows smoothness that looks like a curve and it is differentiable across the entire x axis.
With differentiable we mean that the model can learn and update parameters, so when it is always 0 it means nothing is learned.


### Feed-forward network

Next, here is the code for the feed-forward network. Note that we follow the released code and use the name **multi-layer perceptron (MLP)** rather than “feed-forward network”.

In [6]:
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, config.n_embd * 4)
        self.c_proj = nn.Linear(config.n_embd * 4, config.n_embd)

    def forward(self, x):
        batch_size, seq_len, n_embd = x.shape
        # shape of x [8, 1024, 768] (B, S, E)
        x = self.c_fc(x)
        # shape of x [8, 1024, 3072] because it has been multiplied by 4 (B, S, 4E)
        x = gelu(x)
        # shape of x [8, 1024, 3072] remains the same (B, S, 4E)
        x = self.c_proj(x)
        # shape of x [8, 1024, 768] the input dimension got compressed to it is original dimension (the same as the input) (B, S, E)
        return x
        # shape of x [8, 1024, 768] (B, S, E)

In [7]:
config = Config()
mlp = MLP(config)
batch_size = 8
x = torch.randn( batch_size, config.n_ctx, config.n_embd)
x = mlp.c_fc(x)
print(x.shape)


torch.Size([8, 1024, 3072])


In [8]:
x = gelu(x)
print(x.shape)



torch.Size([8, 1024, 3072])


In [9]:
x = mlp.c_proj(x)
print(x.shape)

torch.Size([8, 1024, 768])


#### 🎓 Task 2.03: Shape annotations

One of the most common errors in deep learning is a mismatch in tensor dimensions. To avoid this, it is good practice to annotate PyTorch code with shapes. For example, suppose you are given the following code:

In [10]:
f = nn.Linear(5, 7)
x = torch.rand(2, 3, 5)
y = f(x)

In [11]:
print(f)

Linear(in_features=5, out_features=7, bias=True)


The annotation of this code with shapes would look as follows:

In [12]:
f = nn.Linear(5, 7)
# not a tensor variable; needs no annotation

x = torch.rand(2, 3, 5)
# shape of x: [2, 3, 5]

y = f(x)
# shape of y: [2, 3, 7]

Annotate the shapes in the `forward()` method of the feed-forward network. Instead of using actual numbers, refer to dimension sizes by symbolic names such as `n_embd`, `batch_size` (number of samples in a batch of input data) and `seq_len` (length of an input sequence). You can introduce additional names and other notation you find useful. Make your annotations as detailed as you need them to explain how the shapes change from one line to the next.

### Causal mask

Our next goal is to implement the core of the GPT architecture: the multi-head attention mechanism.

Recall that the attention mechanism in the Transformer decoder must be restricted to attending only to previously generated tokens. This type of attention is also called **causal attention**. In practice, we implement it through a masking technique that sets the post-softmax attention weights of future tokens to zero. The following utility function implements such a mask:

In [4]:
def make_causal_mask(n):
    return torch.triu(torch.full((n, n), float("-inf")), diagonal=1)

#### 🎈 Task 2.04: Causal mask

Have a close look at the following code and run it to see the result. What are the shapes of `x` and `mask`? Given that the shapes are different, why does the addition operation in the last line not raise an error? What is the shape of the result?

How does the addition operation implement masking? (Recall that the attention scores are normalised using the softmax function.)

In [13]:
x = torch.rand(1, 2, 3, 3)
mask = make_causal_mask(5)
x + mask[:3, :3]

tensor([[[[0.9325,   -inf,   -inf],
          [0.1909, 0.3248,   -inf],
          [0.1767, 0.6797, 0.3953]],

         [[0.9914,   -inf,   -inf],
          [0.0565, 0.4698,   -inf],
          [0.7555, 0.7286, 0.3155]]]])

In [14]:
print(x.shape)
print(mask.shape)
c_mask = x + mask[:3, :3]
print(c_mask.shape)


torch.Size([1, 2, 3, 3])
torch.Size([5, 5])
torch.Size([1, 2, 3, 3])


When comparing 2 shapes, we look at their dimensions and check for 2 rules:


1.   The 2 compared tensors must have the same dimensions size, in this case we have (3, 3) for both x and mask
2.   One of them is 1, in here the mask is only shape of 2 so we fill the missing ones with 1 and it will be [1,1,5,5] **(broadcasting)**. However in the last line we selected only the first 3 from the mask so we get [1,1,3,3],  this will help achieving compatibiality with x [1,2,3,3], otherwise we would get an error.

Now The final result was [1,2,3,3] because when comparing 2 shapes we always take the maximum size of the two input dimensions being compared.

Adding -inf which is the masking is crucial part since it helps minimizing or removing their affect for the prediciton, since we are using softmax, which utilize probability internally, it will see the negative number and will convert it into 0, thus, the model focuses on the current token and all the previous ones since they are part of the input.


### Attention mechanism

Here is the code for the multi-head attention mechanism:

In [15]:
class Attention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.n_head = config.n_head
        self.c_attn = nn.Linear(config.n_embd, config.n_embd * 3)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.register_buffer("mask", make_causal_mask(config.n_ctx), persistent=False)

    def forward(self, x):
        print("Input x:", x.shape)
        batch_size, seq_len, n_embd = x.shape # x.shape: (8, 1024, 768) (B,S,E)
        head_embd = n_embd // self.n_head # head_embd = 768 / 12 = 64 (H)
        q, k, v = self.c_attn(x).chunk(3, dim=-1) # (8, 1024, 768) for all of them (B,S,E)
        print(q.shape, k.shape, v.shape)
        q = q.view(batch_size, seq_len, self.n_head, head_embd) # (8, 1024, 12, 64) (B,S,D,H)
        k = k.view(batch_size, seq_len, self.n_head, head_embd) # (8, 1024, 12, 64) (B,S,D,H)
        v = v.view(batch_size, seq_len, self.n_head, head_embd) # (8, 1024, 12, 64) (B,S,D,H)
        print(q.shape, k.shape, v.shape)
        q = q.transpose(-2, -3) # (8, 12, 1024, 64) (B,D,S,H)
        k = k.transpose(-2, -3) # (8, 12, 1024, 64) (B,D,S,H)
        v = v.transpose(-2, -3) # (8, 12, 1024, 64) (B,D,S,H)
        print(q.shape, k.shape, v.shape)
        x = q @ k.transpose(-1, -2) # (8, 12, 1024, 1024) (B,D,S1,S2)
        print('Transpose x: ', x.shape)
        x = x / head_embd**0.5 # (8, 12, 1024, 1024) (B,D,S1,S2)
        print('x / head_embd**0.5: ', x.shape)
        x = x + self.mask[:seq_len, :seq_len] # (8, 12, 1024, 1024) (B,D,S1,S2)
        print('x + self.mask[:seq_len, :seq_len] x: ', x.shape)
        x = torch.softmax(x, dim=-1) # (8, 12, 1024, 1024) (B,D,S1,S2)
        print('softmax x: ', x.shape)
        x = x @ v # (8, 12, 1024, 64) (B,D,S,H)
        print('x @ v: ', x.shape)
        x = x.transpose(-2, -3).contiguous() # (8, 1024, 12, 64) (B,S,D,H)
        print('Transpose x: ', x.shape)
        x = x.view(batch_size, seq_len, n_embd) #(8, 1024, 768) (B,S,E)
        print('x.view: ', x.shape)
        x = self.c_proj(x) # (8, 1024, 768) (B,S,E)
        print('self.c_proj(x): ', x.shape)
        return x # (B,S,E) which returns an output with the same shape as the initial input

In [16]:
@dataclass
class Config:
    n_vocab: int = 50_257
    n_ctx: int = 1024
    n_embd: int = 768
    n_head: int = 12
    n_layer: int = 12

In [17]:
attention = Attention(config)
batch_size = 8
x = torch.randn(batch_size, config.n_ctx, config.n_embd)
result = attention(x)

Input x: torch.Size([8, 1024, 768])
torch.Size([8, 1024, 768]) torch.Size([8, 1024, 768]) torch.Size([8, 1024, 768])
torch.Size([8, 1024, 12, 64]) torch.Size([8, 1024, 12, 64]) torch.Size([8, 1024, 12, 64])
torch.Size([8, 12, 1024, 64]) torch.Size([8, 12, 1024, 64]) torch.Size([8, 12, 1024, 64])
Transpose x:  torch.Size([8, 12, 1024, 1024])
x / head_embd**0.5:  torch.Size([8, 12, 1024, 1024])
x + self.mask[:seq_len, :seq_len] x:  torch.Size([8, 12, 1024, 1024])
softmax x:  torch.Size([8, 12, 1024, 1024])
x @ v:  torch.Size([8, 12, 1024, 64])
Transpose x:  torch.Size([8, 1024, 12, 64])
x.view:  torch.Size([8, 1024, 768])
self.c_proj(x):  torch.Size([8, 1024, 768])


#### 🎓 Task 2.05: Multi-head attention

Trace the input `x` through the `forward()` method line by line and annotate the shapes of all tensor variables. Identify all lines that rely on broadcasting.

### Layer normalisation

As mentioned above, the inputs to both the feed-forward network and the multi-head attention mechanism undergo **layer normalisation**. This normalises the inputs to have zero mean and unit variance across the activations. [Ba et al. (2016)](https://doi.org/10.48550/arXiv.1607.06450) introduce two trainable parameters (called $\gamma$ and $\beta$ in the paper) that allow the network to learn an appropriate scale and shift for the normalised values.

We implement layer normalisation as follows:

In [18]:
class LayerNorm(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.g = nn.Parameter(torch.ones(config.n_embd))
        self.b = nn.Parameter(torch.zeros(config.n_embd))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        variance = x.var(unbiased=False, dim=-1, keepdim=True)
        return self.g * (x - mean) / torch.sqrt(variance + 1e-05) + self.b

#### 🎈 Task 2.06: Layer normalisation

What is the relevance of the `keepdim=True` keyword argument in the `mean()` and `var()` functions? What would happen if we omitted it?

What is the relevance of the constant 1e-05? What could happen if we omitted it?

`keepdim  (mean)` is used to tell to keep the last dimension in the output and dont remove it when the dimension is reduced, and this will cause mismatch when we do broadcasting since it will be one dimension less.

`keepdim  (var)` is used for the same purpose.

`1e-05` By adding this **epsilon** we are preventing the network from crashing since this helps avoiding division on 0. This is done by adding a very small positive number.

### Decoder block

We now combine the feed-forward network, the multi-head attention mechanism and the layer normalisation into a decoder block.

In [19]:
class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config)
        self.attn = Attention(config)
        self.ln_2 = LayerNorm(config)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

#### 🎓 Task 2.07: Pre-norm and post-norm architectures

The original Transformer ([Vaswani et al., 2017](https://arxiv.org/abs/1706.03762)) is a “post-norm architecture”, where the normalisation is applied **after** each residual block. In contrast, GPT-2 is a “pre-norm architecture”, where the normalisation is applied **before**. Find the passage in Section&nbsp;2.3 of [Radford et al. (2019)](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf) that reports on this modification.

[Xiong et al. (2020)](https://arxiv.org/pdf/2002.04745) compare pre-norm and post-norm architectures empirically. Read the abstract of their paper and summarise their main findings. According to these findings, what are the benefits of the pre-norm architecture?

 - Layer normalization (Ba et al., 2016) was moved to the input of each sub-block, similar to a pre-activation residual network.

 - It provides more stablized version since it prevents the gradients from exploding.
 - It also provides a better perfromace since now the model is more stable even with high learning rate.

### Model

We now have almost all components in place to complete the implementation of the GPT-2 model. The only thing  missing are the position embeddings. These simply associate an embedding vector with every position in the context window. To set them up, we first define another utility function:

In [20]:
def make_positions(n):
    return torch.arange(n, dtype=torch.long)

We then code the complete model as follows:

In [22]:
class Model(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.wte = nn.Embedding(config.n_vocab, config.n_embd)
        self.wpe = nn.Embedding(config.n_ctx, config.n_embd)
        self.h = nn.Sequential(*(Block(config) for _ in range(config.n_layer)))
        self.ln_f = LayerNorm(config)
        self.lm_head = nn.Linear(config.n_embd, config.n_vocab, bias=False)
        self.register_buffer("pos", make_positions(config.n_ctx), persistent=False)

    def forward(self, x):
        batch_size, seq_len = x.shape
        wte = self.wte(x)
        wpe = self.wpe(self.pos[:seq_len]) # type: ignore
        x = wte + wpe
        x = self.h(x)
        x = self.ln_f(x)
        x = self.lm_head(x)
        return x

#### 🎈 Task 2.08: Buffers

Our implementation registers the vector of positions as a buffer. (Earlier, we also registered the causal mask as a buffer.) Consult the PyTorch documentation to determine the benefits of registering a tensor as a buffer, in contrast to computing it in the `forward()` method.

There are 2 reasons :
- 1_ device management: in this case if we want to move the model from the CPU to GPU, Pytorch does this automatically when it sees the buffer and no error will arise.
- 2_ Saving and loading the model based on the last state, this is done also automatically when the Pytorch sees the buffer, in this case non trainable parameters such as (positonal encoding) will be preserved during all the training sessions.

#### 🎓 Task 2.09: Number of trainable parameters

The model we have implemented is the smallest one presented by [Radford et al. (2019)](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf). But how many trainable parameters exactly does it have? Interestingly, the number originally reported by the authors is wrong! What number did they report?

Your task is to write code to compute the number of parameters yourself. This should only take 1–3 lines of code. What number do you get when you apply this code to a fresh model instance?

[Radford et al. (2019)](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf) followed the original Transformers paper ([Vaswani et al., 2017](https://doi.org/10.48550/arXiv.1706.03762)) and shared the trainable weights between the token embedding and the final linear layer. Implement this weight sharing strategy. (Hint: This only requires one line of code.) Then, re-compute the number of trainable parameters for the modified model. What number do you get now? How large is the reduction caused by the weight sharing?

In [24]:
# The number of paramters is `117m`
model = Model(Config())
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of trainable parameters :{total_params}')
model.lm_head.weight = model.wte.weight
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of trainable parameters for the modified model :{total_params}')

Number of trainable parameters :163037184
Number of trainable parameters for the modified model :124439808


## Part 2: Load pre-trained weights

Now that you have a complete implementation of the GPT-2 model in place, you can instantiate it by loading the pre-trained weights released by OpenAI. These weights were originally provided in the TensorFlow format. For this lab, we have re-packaged them as a single file in NumPy’s `.npz` archive format. We can load it as follows:

In [26]:
import numpy as np

pretrained = np.load("gpt-2-pretrained.npz")
print(pretrained)

NpzFile 'gpt-2-pretrained.npz' with keys: h0.attn.c_attn.b, h0.attn.c_attn.w, h0.attn.c_proj.b, h0.attn.c_proj.w, h0.ln_1.b...


The result `pretrained` is a dictionary mapping names to NumPy arrays. When you print the names, you will see that they correspond to the attributes of our network modules, even though the names differ. For example, the array `h0.attn.c_attn.b` holds the biases (`b`) of the `c_attn` linear layer in the attention mechanism (`attn`) of the first transformer block (`h0`).

#### 🎓 Task 2.10: Load pre-trained weights

Create a model from the pre-trained weights. To do this, you need to instantiate a fresh model and write the contents of each array from the `npz` archive with the pre-trained weights into the corresponding tensor. To make this a bit easier, here is a utility function that copies data from a NumPy array `source` to a PyTorch tensor `target`:

In [ ]:
for t in pretrained:
    print(t)

h0.attn.c_attn.b
h0.attn.c_attn.w
h0.attn.c_proj.b
h0.attn.c_proj.w
h0.ln_1.b
h0.ln_1.g
h0.ln_2.b
h0.ln_2.g
h0.mlp.c_fc.b
h0.mlp.c_fc.w
h0.mlp.c_proj.b
h0.mlp.c_proj.w
h1.attn.c_attn.b
h1.attn.c_attn.w
h1.attn.c_proj.b
h1.attn.c_proj.w
h1.ln_1.b
h1.ln_1.g
h1.ln_2.b
h1.ln_2.g
h1.mlp.c_fc.b
h1.mlp.c_fc.w
h1.mlp.c_proj.b
h1.mlp.c_proj.w
h10.attn.c_attn.b
h10.attn.c_attn.w
h10.attn.c_proj.b
h10.attn.c_proj.w
h10.ln_1.b
h10.ln_1.g
h10.ln_2.b
h10.ln_2.g
h10.mlp.c_fc.b
h10.mlp.c_fc.w
h10.mlp.c_proj.b
h10.mlp.c_proj.w
h11.attn.c_attn.b
h11.attn.c_attn.w
h11.attn.c_proj.b
h11.attn.c_proj.w
h11.ln_1.b
h11.ln_1.g
h11.ln_2.b
h11.ln_2.g
h11.mlp.c_fc.b
h11.mlp.c_fc.w
h11.mlp.c_proj.b
h11.mlp.c_proj.w
h2.attn.c_attn.b
h2.attn.c_attn.w
h2.attn.c_proj.b
h2.attn.c_proj.w
h2.ln_1.b
h2.ln_1.g
h2.ln_2.b
h2.ln_2.g
h2.mlp.c_fc.b
h2.mlp.c_fc.w
h2.mlp.c_proj.b
h2.mlp.c_proj.w
h3.attn.c_attn.b
h3.attn.c_attn.w
h3.attn.c_proj.b
h3.attn.c_proj.w
h3.ln_1.b
h3.ln_1.g
h3.ln_2.b
h3.ln_2.g
h3.mlp.c_fc.b
h3.mlp.c_fc.w


In [ ]:
def copy_weights(source: np.ndarray, target: torch.Tensor):
    assert source.shape == target.shape
    with torch.no_grad():
        target.copy_(torch.tensor(source, dtype=torch.float32))

You can start from this skeleton code:

In [ ]:
def from_pretrained() -> Model:
    model = Model(Config())
    pretrained = np.load("gpt-2-pretrained.npz")
    copy_weights(pretrained['wte'], model.wte.weight)
    copy_weights(pretrained['wpe'], model.wpe.weight)
    copy_weights(pretrained['ln_f.b'], model.ln_f.b)
    copy_weights(pretrained['ln_f.g'], model.ln_f.g)
    copy_weights(pretrained['wte'], model.lm_head.weight)
    for i in range(config.n_layer):
      copy_weights(pretrained[f'h{i}.ln_1.b'], model.h[i].ln_1.b)
      copy_weights(pretrained[f'h{i}.ln_1.g'], model.h[i].ln_1.g)
      copy_weights(pretrained[f'h{i}.ln_2.b'], model.h[i].ln_2.b)
      copy_weights(pretrained[f'h{i}.ln_2.g'], model.h[i].ln_2.g)
      copy_weights(pretrained[f'h{i}.attn.c_attn.w'].T, model.h[i].attn.c_attn.weight)
      copy_weights(pretrained[f'h{i}.attn.c_attn.b'], model.h[i].attn.c_attn.bias)
      copy_weights(pretrained[f'h{i}.attn.c_proj.w'].T, model.h[i].attn.c_proj.weight)
      copy_weights(pretrained[f'h{i}.attn.c_proj.b'], model.h[i].attn.c_proj.bias)

      copy_weights(pretrained[f'h{i}.mlp.c_fc.w'].T, model.h[i].mlp.c_fc.weight)
      copy_weights(pretrained[f'h{i}.mlp.c_fc.b'], model.h[i].mlp.c_fc.bias)
      copy_weights(pretrained[f'h{i}.mlp.c_proj.w'].T, model.h[i].mlp.c_proj.weight)
      copy_weights(pretrained[f'h{i}.mlp.c_proj.b'], model.h[i].mlp.c_proj.bias)

    return model

**Important:** One technical detail to note is that PyTorch stores the weights of linear layers in a transposed form. For example, a linear layer created as `nn.Linear(2, 3)` has a weight matrix of shape [3, 2].

## Part 3: Put the model to use

In the third and final part of this lab, you will use the pre-trained model to generate text and evaluate it on a standard benchmark.

### Sampling-based text generation

The easiest way to generate text with a language model is by using a **greedy approach**. This method works by creating text one token at a time. At each step, the model takes the previously generated text (called the **context**) as input and adds the token with the highest output logit as a new token. The code in the next cell defines a function `generate()` that forms the core of a greedy generator:

In [ ]:
def generate(model, context, context_size=1024, n_tokens=20):
    for _ in range(n_tokens):
        context = context[:, -context_size:]
        with torch.no_grad():
            logits = model(context)[:, -1, :]
        next_idx = torch.argmax(logits, dim=-1, keepdim=True)
        context = torch.cat([context, next_idx], dim=-1)
    return context

To use this function with an actual text input, you need a tokeniser to first encode the text into a vector of token IDs, and later decode the generated `context` into new text. The reference implementation of the GPT-2 tokeniser is in the library `tiktoken`. The code in the next cell sets up the tokeniser, loads the pretrained model from Task&nbsp;2.10, and then defines a helper function that handles the encoding and decoding.

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
model = from_pretrained()


def generate_helper(text, context_size=1024, n_tokens=20):
    context = torch.tensor([tokenizer.encode(text)], dtype=torch.long)
    context = generate(model, context, context_size=context_size, n_tokens=n_tokens)
    return tokenizer.decode(context[0].tolist())

:You can use this helper function to generate text as follows:

In [ ]:
generate_helper("Linköping University is")

Input x: torch.Size([1, 5, 768])
torch.Size([1, 5, 768]) torch.Size([1, 5, 768]) torch.Size([1, 5, 768])
torch.Size([1, 5, 12, 64]) torch.Size([1, 5, 12, 64]) torch.Size([1, 5, 12, 64])
torch.Size([1, 12, 5, 64]) torch.Size([1, 12, 5, 64]) torch.Size([1, 12, 5, 64])
Transpose x:  torch.Size([1, 12, 5, 5])
x / head_embd**0.5:  torch.Size([1, 12, 5, 5])
x + self.mask[:seq_len, :seq_len] x:  torch.Size([1, 12, 5, 5])
softmax x:  torch.Size([1, 12, 5, 5])
x @ v:  torch.Size([1, 12, 5, 64])
Transpose x:  torch.Size([1, 5, 12, 64])
x.view:  torch.Size([1, 5, 768])
self.c_proj(x):  torch.Size([1, 5, 768])
Input x: torch.Size([1, 5, 768])
torch.Size([1, 5, 768]) torch.Size([1, 5, 768]) torch.Size([1, 5, 768])
torch.Size([1, 5, 12, 64]) torch.Size([1, 5, 12, 64]) torch.Size([1, 5, 12, 64])
torch.Size([1, 12, 5, 64]) torch.Size([1, 12, 5, 64]) torch.Size([1, 12, 5, 64])
Transpose x:  torch.Size([1, 12, 5, 5])
x / head_embd**0.5:  torch.Size([1, 12, 5, 5])
x + self.mask[:seq_len, :seq_len] x:  to

"Linköping University is a research institute in the Netherlands.\n\nThe research was funded by the European Union's European Research"

**Tip:** If you did not manage to complete Task&nbsp;2.10, you can still work on this task by using a pretrained GPT-2 model from [Hugging Face](https://huggingface.co/openai-community/gpt2). The next code cell shows how you would instantiate this model. Note that you may have to first install the `transformers` library.

In [ ]:
# from transformers import GPT2LMHeadModel
# model = GPT2LMHeadModel.from_pretrained("gpt2")
# logits = model(context).logits[:, -1, :]

#### 🎓 Task 2.11: Sampling-based text generation

 The greedy approach to text generation is not very interesting for practical applications because it always chooses the most likely token, leading to predictable and less creative results. Your task is to modify the code for the `generate()` function to use a **sampling-based approach** instead. In this approach, the next token is chosen randomly based on the probabilities assigned by the model (softmax-normalised logits), treating them as a categorical distribution over the token vocabulary. Additionally, your code should include two common techniques to improve sampling:

 * **temperature scaling**, which lets the user control the randomness of the sampling
 * **top-$k$ sampling**, which limits the sampling to the top-$k$ most likely tokens, ignoring less probable ones

In [ ]:
def generate(model, context, context_size=1024, n_tokens=20, temp=1, top_k=0):
    for _ in range(n_tokens):
        context = context[:, -context_size:]
        with torch.no_grad():
            logits = (model(context)[:, -1, :])/temp
        if top_k > 0:
          # In here we wanted to find the index of the top k values
          top_k_v, _ = torch.topk(logits, k=top_k, dim=-1)
          smallest_v = top_k_v[...,-1].unsqueeze(-1)
          # In here are are mask all the logits below the smallest k value by setting it negative inf to not affect the prediciton
          logits = torch.where(logits < smallest_v, torch.tensor(float('-inf')), logits)
        prob = torch.softmax(logits,dim=-1)
        next_index = torch.multinomial(prob, num_samples=1)
        context = torch.cat([context, next_index], dim=-1)
    return context

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
model = from_pretrained()


def generate_helper(text, context_size=1024, n_tokens=20):
    context = torch.tensor([tokenizer.encode(text)], dtype=torch.long)
    context = generate(model, context, context_size=context_size, n_tokens=n_tokens)
    return tokenizer.decode(context[0].tolist())

In [ ]:
generate_helper("Linköping University is")

Input x: torch.Size([1, 5, 768])
torch.Size([1, 5, 768]) torch.Size([1, 5, 768]) torch.Size([1, 5, 768])
torch.Size([1, 5, 12, 64]) torch.Size([1, 5, 12, 64]) torch.Size([1, 5, 12, 64])
torch.Size([1, 12, 5, 64]) torch.Size([1, 12, 5, 64]) torch.Size([1, 12, 5, 64])
Transpose x:  torch.Size([1, 12, 5, 5])
x / head_embd**0.5:  torch.Size([1, 12, 5, 5])
x + self.mask[:seq_len, :seq_len] x:  torch.Size([1, 12, 5, 5])
softmax x:  torch.Size([1, 12, 5, 5])
x @ v:  torch.Size([1, 12, 5, 64])
Transpose x:  torch.Size([1, 5, 12, 64])
x.view:  torch.Size([1, 5, 768])
self.c_proj(x):  torch.Size([1, 5, 768])
Input x: torch.Size([1, 5, 768])
torch.Size([1, 5, 768]) torch.Size([1, 5, 768]) torch.Size([1, 5, 768])
torch.Size([1, 5, 12, 64]) torch.Size([1, 5, 12, 64]) torch.Size([1, 5, 12, 64])
torch.Size([1, 12, 5, 64]) torch.Size([1, 12, 5, 64]) torch.Size([1, 12, 5, 64])
Transpose x:  torch.Size([1, 12, 5, 5])
x / head_embd**0.5:  torch.Size([1, 12, 5, 5])
x + self.mask[:seq_len, :seq_len] x:  to

'Linköping University is still impressing international media this week with its discovery that Neanderthal bones from Pembroke P'

### Evaluating the pretrained model

If you have experimented with your pretrained GPT-2 model, you will have noticed that its ability to generate useful text is somewhat limited. By today’s standards, GPT-2 is a small model with modest capabilities. However, it can still be helpful for certain tasks, such as text autocompletion, generating filler text, or answering simple questions. To rigourosly evaluate language models, researchers often use standard benchmark datasets. Creating these benchmarks is a discipline of its own, and they tend to become increasingly challenging as models continue to improve.

In the final task of this lab, you will evaluate GPT-2’s performance on a small subset of the [HellaSwag dataset](https://rowanzellers.com/hellaswag/), which was published in the same year as GPT-2 itself (2019). HellaSwag is designed to test a model’s ability to perform commonsense reasoning in challenging contexts. Unlike simpler benchmarks, HellaSwag presents scenarios where the correct text completion depends on semantic relationships between events and on world knowledge. This makes it a good choice for assessing the ability of language models to go beyond surface-level patterns and produce meaningful, context-aware predictions.

#### 🎓 Task 2.12: Evaluating the pretrained model

Read the [HellaSwag website](https://rowanzellers.com/hellaswag/) to get some background on the benchmark. How does a sample from the dataset look like? What is an expected prediction? How does the benchmark allow us to score models? What is the random baseline? What is the human performance reported on the task?

The next cell contains code for evaluating your pretrained model on a small sample from HellaSwag. You will also need a tokenizer. The HellaSwag subset is in the file `hellaswag-mini.jsonl`. Inspect that file to understand the format. Next, read the code and explain how it works. Specifically, how does the code compute the score of individual endings? In the call to `cross_entropy()`, why are the tensors sliced in this specific way?

Finally, what overall score does the pretrained GPT-2 model get on this benchmark? How does that score compare to the random baseline and the human performance?

In [ ]:
import json

with open("hellaswag-mini.jsonl") as f:
    n_correct = 0 # number of correctly predicted samples.
    n_total = 0 # total number of samples processed.
    for line in f:
        sample = json.loads(line) # Parses the current JSON line into a Python dictionary 'sample'.
        prefix = tokenizer.encode(sample["ctx"]) # Tokenizes the context.
        ending_scores = []
        for i, ending in enumerate(sample["endings"]): # Iterates through the list of endings for the context.
            suffix = tokenizer.encode(" " + ending) # Tokenizes the current ending.
            context = torch.tensor([prefix + suffix], dtype=torch.long) # Combines prefix and suffix tokens.
            with torch.no_grad(): # disables gradient calculation (to save resources).
                logits = model(context) # Generating logits.
                ending_score = torch.nn.functional.cross_entropy( # Calculates the negative log likelihood as the score for this ending.
                    logits[0, -len(suffix) - 1 : -1], context[0, -len(suffix) :] # Compares the logits against the actual suffix tokens.
                )
            ending_scores.append((ending_score, i)) # Stores the calculated score (loss) and the index 'i' of the current ending.
        predicted = min(ending_scores)[1] # fetching the index of the ending with the minimum score.
        n_correct += int(predicted == sample["label"])
        n_total += 1
    print(f"Accuracy: {n_correct / n_total:.2%}")

Streaming output truncated to the last 5000 lines.
x / head_embd**0.5:  torch.Size([1, 12, 116, 116])
x + self.mask[:seq_len, :seq_len] x:  torch.Size([1, 12, 116, 116])
softmax x:  torch.Size([1, 12, 116, 116])
x @ v:  torch.Size([1, 12, 116, 64])
Transpose x:  torch.Size([1, 116, 12, 64])
x.view:  torch.Size([1, 116, 768])
self.c_proj(x):  torch.Size([1, 116, 768])
Input x: torch.Size([1, 116, 768])
torch.Size([1, 116, 768]) torch.Size([1, 116, 768]) torch.Size([1, 116, 768])
torch.Size([1, 116, 12, 64]) torch.Size([1, 116, 12, 64]) torch.Size([1, 116, 12, 64])
torch.Size([1, 12, 116, 64]) torch.Size([1, 12, 116, 64]) torch.Size([1, 12, 116, 64])
Transpose x:  torch.Size([1, 12, 116, 116])
x / head_embd**0.5:  torch.Size([1, 12, 116, 116])
x + self.mask[:seq_len, :seq_len] x:  torch.Size([1, 12, 116, 116])
softmax x:  torch.Size([1, 12, 116, 116])
x @ v:  torch.Size([1, 12, 116, 64])
Transpose x:  torch.Size([1, 116, 12, 64])
x.view:  torch.Size([1, 116, 768])
self.c_proj(x):  torch.

**HellaSwag** is used to test the model's ability in understanding the context, this is done by using a methods called Adversial filtering that generate a correct ending and multiple of worng ends to trick the models, this technique showed that the models found hard time selecting the correct answer with accuracay dropped from 86.7% to 46% for state of the art model called BERT.

**The expected predictions** is the follow up sentence that complete the context
The models scores is based on their accuracy based on their selection of the 4 options that is been generated to trick the model. and then the result is compared with humans (they reached 95.6% accuracy) and random baseline (28% accuracy), the random baseline is a techniuqe that is similar to a technique I used to do whenever I have a multiple choice qustion and I have no clue about the answer, so in this case just pick one random option.

The scoring for the ending is done through Cross-Entropy Loss, which measures how well the model predicted the sequence of tokens that built the ending.
**Slicing Rationale** it aligns the logits with the true tokens by shifting the logits one position to the left.

**🥳 Congratulations on finishing lab&nbsp;2!**